# Лабораторная работа №1. Крестики - нолики

*Арешин Станислав Олегович М8О-211M-21*


**Обучаем агента играть против случайной стратегии**

* Выгрыш: 1

* Проигрыш: -1

* Ничья: 0

* Ход в ту же клетку: -2

**Поле: одномерный массив длины 9, позиция 0 - верхний левый угол, позиция 8 - нижний правый угол по порядку соответственно**

In [1]:
import gym
import numpy as np
import stable_baselines3

In [2]:
class MyEnv(gym.Env):
    '''
    action_space: возможные позиции для хода 0-8
    observation_space: словарь с двумя бинарными векторами с занятыми позициями для каждого игрока отдельно
        1 - занята, 0 - свободна
    '''
    def __init__(self):
        super().__init__()
        self.action_space = gym.spaces.Discrete(3 * 3)
        self.observation_space = gym.spaces.Dict({
            'agent': gym.spaces.MultiBinary(3 * 3), 
            'strategy': gym.spaces.MultiBinary(3 * 3)
        })   
    
    '''
    Обнуляем доску - все позиции свободны
    '''
    def reset(self):
        self.state = {'agent': np.zeros(3 * 3), 'strategy': np.zeros(3 * 3)}
        return self.state
    
    '''
    Агент ходит первым
    Если он пытается сходить в занятую позицию: штраф -2 и заканчиваем ход
    В случае выгрыша: поощряем +1, заканчиваем игру 
    Если ничья: 0, заканчиваем игру 
    Далее ходит случайная стратегия на любую свободную позицию
    Если агент проигрывает после хода противника: штраф -1, заканчиваем игру 
    Ничья: 0, заканчиваем игру 
    Игра не закончена: 0, следующий ход
    '''
    def step(self, action):
        self.reward = 0
        # проверяем на ход в занятую позицию
        if self.state['strategy'][action] == 1 or self.state['agent'][action] == 1:
            self.reward  = -2
            return self.state, self.reward, self.done, {}
        # действие агента     
        self.state['agent'][action] = 1 
        # проверяем победу агента
        self.check_win('agent')
        if self.done:
            self.reward = 1
            return self.state, self.reward, self.done, {}
        # проверяем ничью
        if ~np.any((self.state['agent'] + self.state['strategy']) == 0):
            self.done = True
            return self.state, self.reward, self.done, {}
        
        # действие случайной стратегии
        player_action = self.get_player_action()
        self.state['strategy'][player_action] = 1
        # проверяем победу стратегии
        self.check_win('strategy')
        if self.done:
            self.reward  = -1
            return self.state, self.reward, self.done, {}
        # проверяем ничью
        if ~np.any((self.state['agent'] + self.state['strategy']) == 0):
            self.done = True
            return self.state, self.reward, self.done, {}
        
        return self.state, self.reward, self.done, {}
    
    '''
    Действие случайной стратегии, выбираем любую свободную позицию
    '''
    def get_player_action(self):
        return np.random.choice(np.argwhere((self.state['agent'] + self.state['strategy']) == 0).reshape(-1))
            
    
    '''
    Проверка победы игрока
    '''
    def check_win(self, player):
        self.done = False
        if np.sum(self.state[player][:3]) == 3 \
            or np.sum(self.state[player][3:6]) == 3 \
            or np.sum(self.state[player][6:]) == 3 \
            or self.state[player][0] + self.state[player][3] + self.state[player][6] == 3 \
            or self.state[player][1] + self.state[player][4] + self.state[player][7] == 3 \
            or self.state[player][2] + self.state[player][5] + self.state[player][8] == 3 \
            or self.state[player][0] + self.state[player][4] + self.state[player][8] == 3 \
            or self.state[player][2] + self.state[player][4] + self.state[player][6] == 3:
                
            self.done = True 

In [3]:
%%time
# инициализация среды и обучение алгоритма 
env = MyEnv()
model = stable_baselines3.PPO("MultiInputPolicy", env, verbose=False)
model.learn(total_timesteps=20000)

Wall time: 53.3 s


In [4]:
# играем

obs = env.reset()
actions = []
done = False

while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    actions.append(action)
    print(obs, done)
    
if reward == 1:
    print('Win')
elif reward == -1: 
    print('Lose')
elif reward == 0:
    print('Draw')
else:
    print('Same step')

print(actions)

env.close()

{'agent': array([0., 0., 0., 0., 1., 0., 0., 0., 0.]), 'strategy': array([0., 0., 1., 0., 0., 0., 0., 0., 0.])} False
{'agent': array([0., 0., 0., 0., 1., 0., 1., 0., 0.]), 'strategy': array([0., 0., 1., 0., 0., 1., 0., 0., 0.])} False
{'agent': array([0., 0., 0., 0., 1., 0., 1., 1., 0.]), 'strategy': array([0., 0., 1., 1., 0., 1., 0., 0., 0.])} False
{'agent': array([0., 0., 0., 0., 1., 0., 1., 1., 1.]), 'strategy': array([0., 0., 1., 1., 0., 1., 0., 0., 0.])} True
Win
[4, 6, 7, 8]


Если обучить недостаточно, агент может проиграть даже случайной стратегии, либо зациклиться на одном и том же ходе, ничью не встречал. Увеличив время обучения, получаем агента, который всегда обыгрывает случайную стратегию.